# Final Project App

#### By Vaishnavi Pulla

Please make sure the csv file is in the same folder as the program. The csv file used should be the one attached in the submission.

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
pip install numpy matplotlib pandas seaborn scikit-learn torch shiny

Note: you may need to restart the kernel to use updated packages.


In [7]:
!pip install shiny nest_asyncio
import nest_asyncio
nest_asyncio.apply()

In [ ]:
#WORKS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from shiny import App, render, ui

dat = pd.read_csv("liverdisease.csv")
user_input = []

# Handle missing values
dat.fillna(dat.median(), inplace=True)

X = dat.iloc[:, 0:10]  # First 10 columns are features
y = dat["Cirrhosis"]   # Target variable

# Scaling features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Training Logistic Model
logreg = LogisticRegression(random_state=16, max_iter=500, class_weight='balanced', solver='liblinear')
logreg.fit(X_scaled, y)



#NEURAL NET
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, classification_report, confusion_matrix
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

class NeuralNet4L(nn.Module):
    def __init__(self, input_size):
        super(NeuralNet4L, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.rrelu = nn.RReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)
    def forward(self, x):
        x = self.rrelu(self.fc1(x))
        x = self.rrelu(self.fc2(x))
        return self.fc3(x)

scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tensor = torch.FloatTensor(X_train)
y_train_tensor = torch.FloatTensor(y_train.values).view(-1, 1)
X_test_tensor = torch.FloatTensor(X_test)
y_test_tensor = torch.FloatTensor(y_test.values).view(-1, 1)
user_input_tensor = torch.FloatTensor(user_input).view(-1,1)

model = NeuralNet4L(input_size=X_train.shape[1])
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.02)
num_epochs = 1000

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

model.eval()

# Define the UI
app_ui = ui.page_fluid(
    ui.h1("Cirrhosis Self-Diagnosis"),
    ui.layout_sidebar(
        ui.sidebar(
            ui.card_header("Input your Lab Results and Information Below"),
            ui.input_selectize("sex", "Choose your Sex", ["Male", "Female"]),
            ui.input_slider("age_ui", "Choose your Age", 0, 100, 50),
            ui.input_numeric("DB_ui", "Enter your Direct Bilirubin", 1.4, min=0.1, max=20),
            ui.input_numeric("AP_ui", "Enter your Alkaline Phosphotase", 291, min=60, max=2200),
            ui.input_numeric("Sgpt_ui", "Enter your Alanine Aminotransferase", 81, min=10, max=2000),
            ui.input_numeric("Sgot_ui", "Enter your Aspartate Aminotransferase", 10, min=10, max=5000),
            ui.input_numeric("TP_ui", "Enter your Total Proteins", 6.5, min=2.5, max=10),
            ui.input_numeric("AG_Ratio_ui", "Enter your Albumin/Globulin Ratio", 0.95, min=0.3, max=3),
            ui.input_action_button("action_btn", "Submit")  # Action button
        ),
            ui.h3("Results and Graphs"),
            ui.p("Disclaimer: These results are not intended to be used in replacement of consulting a doctor."),
            ui.output_text("result"),
            ui.output_plot("plt1", width='75%', height='200px'),
            ui.output_plot("plt2", width = '75%', height= '200px'),
            ui.output_plot("plt3", width = '75%', height= '200px'),
            ui.output_plot("plt4", width = '75%', height= '200px'),
            ui.output_plot("plt5", width = '75%', height= '200px'),  
            ui.output_plot("plt6", width = '75%', height= '200px')
    )
)

# Define the server logic
def server(input, output, session):
    @output
    @render.text
    def result():
        if input.action_btn() == 0:
            return "Please enter your details and press Submit."
        
        sex_recode = 1 if input.sex() == "Female" else 2 if input.sex() == "Male" else None
        
        # Collect user inputs
        user_input = [
            input.age_ui(),
            sex_recode,
            dat['Total Bilirubin'].median(),  # Total Bilirubin
            input.DB_ui(),                    # Direct Bilirubin
            input.AP_ui(),                    # Alkaline Phosphatase
            input.Sgpt_ui(),                  # Alanine Aminotransferase
            input.Sgot_ui(),                  # Aspartate Aminotransferase
            input.TP_ui(),                    # Total Proteins
            dat['Albumin'].median(),          # Albumin
            input.AG_Ratio_ui()               # Albumin/Globulin Ratio
        ]

        # Check for missing values
        if None in user_input:
            return "Error: All inputs must be provided."

        # Scale the user input with the same scaler used for training
        user_input_array = np.array(user_input).reshape(1, -1)
        user_scaled = scaler.transform(user_input_array)

        # Predict using the logistic regression model
        y_pred_LR= logreg.predict(user_scaled)

        user_scaled_tensor = torch.FloatTensor(user_scaled)
        
        with torch.no_grad():
          y_pred_NN = model(user_scaled_tensor).numpy()

        NN_result = 1 if y_pred_NN > 0.5 else 0
        
        results = [y_pred_LR, NN_result]
        

        if results[0] and results[1] == 0:
            return  "Based on your input, it is unlikely that you are at risk for cirrhosis."
        elif results[0] and results[1] == 1:
            return "Based on your input, it is likely that you are at risk for cirrhosis."
        else:
            return "Based on conflicting results from the neural network and logistic regression, it is inconclusive whether you are at risk for cirrhosis."
            

    @output
    @render.plot
    def plt1():
        fig, ax = plt.subplots()
        ax.hist(dat["Direct Bilirubin"], bins=30, color="grey", alpha=0.7)
        if input.DB_ui() is not None:
            ax.axvline(input.DB_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Direct Bilirubin Levels in the Population")
        ax.set_xlabel("Grams per Deciliter")
        ax.set_ylabel("Frequency")
        return fig

    @output
    @render.plot
    def plt2():
        fig, ax = plt.subplots()
        ax.hist(dat["Alkaline Phosphotase"], bins=30, color="grey", alpha=0.7)
        if(input.AP_ui() is not None):
            ax.axvline(input.AP_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Alkaline Phosphatase Levels in the Population")
        ax.set_xlabel("Milligrams per Deciliter")
        ax.set_ylabel("Frequency")
        return fig


    @output
    @render.plot
    def plt3():
        fig, ax = plt.subplots()
        ax.hist(dat["Alamine Aminotransferase"], bins=30, color="grey", alpha=0.7)
        if(input.Sgpt_ui() is not None):
            ax.axvline(input.Sgpt_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Alanine Aminotransferase Levels in the Population")
        ax.set_xlabel("U/L")
        ax.set_ylabel("Frequency")
        return fig

    @output
    @render.plot
    def plt4():
        fig, ax = plt.subplots()
        ax.hist(dat["Aspartate Aminotransferase"], bins=30, color="grey", alpha=0.7)
        if(input.Sgpt_ui() is not None):
            ax.axvline(input.Sgot_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Aspartate Aminotransferase Levels in the Population")
        ax.set_xlabel("U/L")
        ax.set_ylabel("Frequency")
        return fig

    @output
    @render.plot
    def plt5():
        fig, ax = plt.subplots()
        ax.hist(dat["Total Proteins"], bins=30, color="grey", alpha=0.7)
        if(input.Sgpt_ui() is not None):
            ax.axvline(input.TP_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Total Proteins Levels in the Population")
        ax.set_xlabel("U/dL")
        ax.set_ylabel("Frequency")
        return fig

    @output
    @render.plot
    def plt6():
        fig, ax = plt.subplots()
        ax.hist(dat["Albumin and Globulin Ratio"], bins=30, color="grey", alpha=0.7)
        if(input.Sgpt_ui() is not None):
            ax.axvline(input.AG_Ratio_ui(), color="red", linestyle="dashed", linewidth=2, label="Your Levels")
            ax.legend(loc="upper right")
        ax.set_title("Albumin and Globulin Ratio in the Population")
        ax.set_ylabel("Frequency")
        return fig

# Create the Shiny app
app = App(app_ui, server)

from shiny import run_app
await run_app(app)

INFO:     Started server process [69183]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:60493 - "GET / HTTP/1.1" 200 OK


INFO:     ('127.0.0.1', 60495) - "WebSocket /websocket/" [accepted]
INFO:     connection open
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/